In [1]:
import os
os.environ['MXNET_ENGINE_TYPE'] = 'NaiveEngine'
FIG_SIZE = (12, 8)
import numpy as np
N = 50
X = np.linspace(-5, 5, N)[:, None]
Y = (X > 0).astype(float)

In [2]:
import matplotlib.pyplot as plt
plt.figure(figsize=FIG_SIZE)
plt.title('Step Function')
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.plot(X, Y);

In [3]:
import mxfusion
import numpy as np
from mxfusion.common import config
config.DEFAULT_DTYPE = 'float64'

In [4]:
import mxnet as mx
import mxfusion.components.distributions.gp.kernels
from mxfusion.components.variables import PositiveTransformation
from mxfusion.modules.gp_modules.dgp import DeepGPRegression

M = 10
D = 1
z_locations = np.linspace(-5, 5, 10)[:, None]
m = mxfusion.Model()
m.N = mxfusion.Variable()
m.X = mxfusion.Variable(shape=(m.N, D))
m.Z_0 = mxfusion.Variable(shape=(M, D), initial_value=mx.nd.array(z_locations))
#m.Z_1 = mxfusion.Variable(shape=(M, D), initial_value=mx.nd.array(z_locations))

# this is the likelihood noise at the final layer
m.noise_var = mxfusion.Variable(transformation=PositiveTransformation())

kernels = [mxfusion.distributions.gp.kernels.RBF(D)]
m.Y = DeepGPRegression.define_variable(m.X, kernels, m.noise_var, shape=(m.N, 1), 
                                       inducing_inputs=[m.Z_0],#, m.Z_1], 
                                       n_samples=5)



In [5]:
import mxnet as mx
from mxfusion.inference import GradBasedInference, MAP

infr = GradBasedInference(inference_algorithm=MAP(model=m, observed=[m.X, m.Y]))
infr.run(X=mx.nd.array(X, dtype='float64'), Y=mx.nd.array(Y, dtype='float64'), max_iter=1, 
         learning_rate=0.05, verbose=True)

infr.params[m.Y.factor._module_graph.noise_var] = mx.nd.array([1e-6])

infr.run(X=mx.nd.array(X, dtype='float64'), Y=mx.nd.array(Y, dtype='float64'), max_iter=2000, 
         learning_rate=0.05, verbose=True)

/Users/marpulli/git/MXFusion/mxfusion/inference/inference_parameters.py:74: UserWarning: InferenceParameters has already been initialized.  The existing one will be overwritten.
  warnings.warn("InferenceParameters has already been initialized.  The existing one will be overwritten.")


cov (1, 50, 50)
(1, 1, 50, 10)


NameError: name 'MultivariateNormal' is not defined

In [ ]:
infr.run(X=mx.nd.array(X, dtype='float64'), Y=mx.nd.array(Y, dtype='float64'), max_iter=2000, 
         learning_rate=0.05, verbose=True)

In [ ]:
from mxfusion.inference import ModulePredictionAlgorithm, TransferInference
xt = np.linspace(-10, 10, 100)[:, None]

# More samples to give smoother plots
m.Y.factor.dgp_predict.n_samples = 500
infr_pred = TransferInference(ModulePredictionAlgorithm(model=m, observed=[m.X], target_variables=[m.Y]), 
                              infr_params=infr.params)

res = infr_pred.run(X=mx.nd.array(xt, dtype='float64'))[0]
f_mean, f_var = res[0].asnumpy()[0], res[1].asnumpy()[0]

import matplotlib.pyplot as plt
plt.figure(figsize=(12, 8))

mean = res[0].asnumpy()[:, 0]
plt.plot(xt, res[0].asnumpy().T)

std = np.sqrt(res[1].asnumpy())[:, 0]
plt.fill_between(xt[:, 0], mean - std, mean + std, alpha=0.1)
plt.scatter(X, Y.T)

In [ ]:
infr.params[m.Y.factor._extra_graphs[0].qU_mean_0] = mx.nd.array([[0, 0, 0, 0, 0, 1, 1, 1, 1, 1]])

In [ ]:
infr.params[m.Y.factor._extra_graphs[0].qU_mean_1] = mx.nd.array([[0, 0, 0, 0, 0, 1, 1, 1, 1, 1]])

In [ ]:
infr.params[kernels[0].variance]

In [ ]:
infr.params[kernels[1].variance]

In [ ]:
plt.figure()
plt.plot(infr.params[m.Y.factor._extra_graphs[0].inducing_points_0])